# Τι Καλό Υπάρχει στην Τηλεόραση;

* Προσαρμοσμένο το από άρθρο του The Economist [για την ανάλυση των τηλεοπτικών σειρών στην Αμερική](https://www.economist.com/graphic-detail/2018/11/24/tvs-golden-age-is-real).

---

> Πάνος Λουρίδας, Αναπληρωτής Καθηγητής <br />
> Τμήμα Διοικητικής Επιστήμης και Τεχνολογίας <br />
> Οικονομικό Πανεπιστήμιο Αθηνών<br />
> louridas@aueb.gr

* Για την ανάλυσή μας θα χρησιμοποιήσουμε τα στοιχεία που δίνει το [IMDb](https://www.imdb.com).

* Αυτά είναι διαθέσιμα από το <https://www.imdb.com/interfaces/>.

* Μπορείτε να τα προμηθευτείτε ελεύθερα από εκεί (δεν μπορούμε να τα διαθέσουμε εμείς εδώ), ώστε να αναπαράξετε τη συνέχεια.

* Θεωρούμε ότι τα έχετε προμηθευτεί και τα έχετε αποθηκεύσει στον κατάλογο (φάκελο) `tvseries`.

In [1]:
import pandas as pd
import csv
import numpy as np

import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

* Η βασική πληροφορία για ταινίες και τηλεοπτικές σειρές περιέχεται στο αρχείο `title.basics.tsv.gz`.

* Είναι μεγάλο αρχείο, και το pandas θα παραπονεθεί αν δεν δώσουμε τους τύπους δεδομένων των στηλών. 

In [2]:
column_types = {
    'isAdult': float,
    'startYear': float,
    'endYear': float,
    'runtimeMinutes': float,
    'tconst': str,
    'titleType': str,
    'primaryTitle': str,
    'originalTitle': str,
    'genres': str
}

titles_df = pd.read_csv("tvseries/title.basics.tsv.gz", 
                        dtype=column_types,
                        na_values=r'\N',
                        sep="\t",
                        quoting=csv.QUOTE_NONE)

* Ιδού πώς είναι τα δεδομένα:

In [3]:
print(titles_df.shape)
titles_df.head()

(8686339, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


* Μπορεί να αναρωτιέστε γιατί δώσαμε την παράμετρο `quoting=csv.QUOTE_NONE`.

* Το κάναμε αυτό γιατί τα δεδομένα έχουν κάποια θέματα με την μορφοποίησή τους.

* Πώς το βρήκαμε αυτό;

* Με άσχημο τρόπο. Το `read_csv()` έσκαγε, οπότε γράψαμε μια παραλλαγή της δυαδικής αναζήτησης για να βρούμε σε ποια ακριβώς γραμμή εμφανιζόταν το πρόβλημα.

* Για να τρέξει η δυαδική αναζήτηση χρειάζεται να ξέρουμε τον αριθμό γραμμών του αρχείου, που τον βρίσκουμε με:

```
gunzip -c title.basics.tsv.gz | wc -l
```

In [4]:
high = !gunzip -c tvseries/title.basics.tsv.gz | wc -l
high = int(high[0]) # The ! magic command returns a list-like type
high -= 1 # don't count the header
high

8686339

* Και να η παραλλαγή της δυαδικής αναζήτησης:

In [5]:
colnames = [
    'tconst',
    'titleType', 
    'primaryTitle',
    'originalTitle',    
    'isAdult',
    'startYear',
    'endYear',
    'runtimeMinutes',
    'genres'
]

def binary_search_error(low, high):
    print(low, '<= check <=', high)
    if low > high:
        return -1
    try:
        titles_df = pd.read_csv("tvseries/title.basics.tsv.gz",
                                header=None,
                                names=colnames,
                                dtype=column_types,
                                na_values=r'\N',
                                sep="\t",
                                skiprows=low,
                                nrows=high - low + 1)
    except ValueError as ver:
        if low == high:
            return low + 1 # add header offset
        mid = low + (high - low) // 2
        found = binary_search_error(low, mid)
        if found == -1:
            return binary_search_error(mid+1, high)
        else:
            return found
    return -1
        
error_line = binary_search_error(1, high)
print('found at:', error_line)

1 <= check <= 8686339
1 <= check <= 4343170
1 <= check <= 2171585
1 <= check <= 1085793
1085794 <= check <= 2171585
1085794 <= check <= 1628689
1085794 <= check <= 1357241
1085794 <= check <= 1221517
1085794 <= check <= 1153655
1085794 <= check <= 1119724
1085794 <= check <= 1102759
1085794 <= check <= 1094276
1094277 <= check <= 1102759
1094277 <= check <= 1098518
1094277 <= check <= 1096397
1096398 <= check <= 1098518
1096398 <= check <= 1097458
1097459 <= check <= 1098518
1097459 <= check <= 1097988
1097989 <= check <= 1098518
1097989 <= check <= 1098253
1098254 <= check <= 1098518
1098254 <= check <= 1098386
1098254 <= check <= 1098320
1098254 <= check <= 1098287
1098254 <= check <= 1098270
1098254 <= check <= 1098262
1098263 <= check <= 1098270
1098263 <= check <= 1098266
1098267 <= check <= 1098270
1098267 <= check <= 1098268
1098269 <= check <= 1098270
1098269 <= check <= 1098269
1098270 <= check <= 1098270
found at: 1098271


* Τώρα που εντοπίσαμε προβληματική γραμμή, αν την εξετάσουμε θα δούμε ότι το πρόβλημα παρουσιάζεται από ένα εισαγωγικό σε ένα πεδίο:

In [6]:
titles_df.iloc[error_line]

tconst                    tt1023336
titleType                     movie
primaryTitle      Where's My Stuff?
originalTitle     Where's My Stuff?
isAdult                         0.0
startYear                    2011.0
endYear                         NaN
runtimeMinutes                 85.0
genres                       Comedy
Name: 1098271, dtype: object

* Μας ενδιαφέρουν μόνο οι τηλεοπτικές σειρές, άρα ας δούμε τι είδους οντότητες περιγράφει η στήλη `titleType`.

In [7]:
titles_df['titleType'].unique()

array(['short', 'movie', 'tvEpisode', 'tvSeries', 'tvShort', 'tvMovie',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

* Εμείς θα κρατήσουμε αυτές που έχουν σχέση με τηλεόραση.

In [8]:
tv_types = [
    'tvMovie',
    'tvSeries',
    'tvEpisode',
    'tvShort',
    'tvMiniSeries',
    'tvSpecial'
]
titles_df = titles_df.loc[titles_df['titleType'].isin(tv_types)]

* Θα κρατήσουμε τις σειρές στα αγγλικά (ή τουλάχιστον αυτές που ο τίτλος τους είναι ο ίδιο με τον πρωτότυπο τίτλο τους:

In [9]:
titles_df = titles_df.loc[titles_df['primaryTitle'] == titles_df['originalTitle']]
titles_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
20354,tt0020666,tvEpisode,Barnacle Bill,Barnacle Bill,0.0,1930.0,NaN,8.0,"Animation,Comedy,Family"
20515,tt0020829,tvEpisode,Dizzy Dishes,Dizzy Dishes,0.0,1930.0,NaN,6.0,"Animation,Comedy,Family"
20840,tt0021166,tvEpisode,Mysterious Mose,Mysterious Mose,0.0,1930.0,NaN,6.0,"Animation,Comedy,Family"
21270,tt0021612,tvEpisode,Any Little Girl That's a Nice Little Girl,Any Little Girl That's a Nice Little Girl,0.0,1931.0,NaN,7.0,"Animation,Comedy,Family"
21313,tt0021655,tvEpisode,Betty Co-ed,Betty Co-ed,0.0,1931.0,NaN,6.0,"Animation,Comedy,Family"
...,...,...,...,...,...,...,...,...,...
8686333,tt9916846,tvEpisode,Episode #3.18,Episode #3.18,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
8686334,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
8686335,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
8686336,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0.0,2010.0,NaN,NaN,"Action,Drama,Family"


* Αφού θα εξετάσουμε την εξέλιξη των σειρών στη διάρκεια του χρόνου, να δούμε ποια είναι αυτή η διάρκεια:

In [10]:
titles_df.loc[titles_df['startYear'].idxmin()]

tconst                          tt9001916
titleType                        tvSeries
primaryTitle      Grand Prix Motor Racing
originalTitle     Grand Prix Motor Racing
isAdult                               0.0
startYear                          1906.0
endYear                            1949.0
runtimeMinutes                        NaN
genres                              Sport
Name: 8261427, dtype: object

* Αυτό δεν στέκει, αφού δεν υπήρχε τηλεόραση το 1906.

* Ας θεωρήσουμε κατ' αρχήν ότι έκτοπες τιμές έχουμε πέρα από το 10%.

In [11]:
titles_df['startYear'].quantile(.01)

1957.0

* Για να είμαστε λίγο πιο γενναιώδοροι, θα κρατήσουμε το υλικό από το 1945 και μετά. 

In [12]:
titles_df = titles_df[titles_df['startYear'] >= 1945]

* Συνεχίζουμε διαβάζοντας τα στοιχεία των επεισοδίων.

In [13]:
column_types = {
    'seasonNumber': float,
    'episodeNumber': float,
    'tconst': str,
    'parentTconst': str
}

episodes_df = pd.read_csv("tvseries/title.episode.tsv.gz", 
                          dtype=column_types,
                          na_values=r'\N',
                          sep="\t",
                          quoting=csv.QUOTE_NONE)

* Ιδού πώς είναι τα επεισόδια:

In [14]:
print(episodes_df.shape)
episodes_df.head()

(6496963, 4)


,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0020666,tt15180956,1.0,2.0
1,tt0020829,tt15180956,1.0,1.0
2,tt0021166,tt15180956,1.0,3.0
3,tt0021612,tt15180956,2.0,2.0
4,tt0021655,tt15180956,2.0,5.0


* Θα ενώσουμε τα επεισόδια με το `DataFrame` `title_df`, ώστε να έχουμε όλη την πληροφορία για κάθε επεισόδιο.

In [15]:
titles_episodes_df = pd.merge(titles_df, episodes_df, 
                              left_on='tconst', 
                              right_on='parentTconst',
                              suffixes=['_ti', '_ep'])
titles_episodes_df.shape

(5968836, 13)

* Το αποτέλεσμα:

In [16]:
titles_episodes_df.head()

,tconst_ti,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tconst_ep,parentTconst,seasonNumber,episodeNumber
0,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt12257020,tt0038276,NaN,NaN
1,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt12283504,tt0038276,NaN,NaN
2,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt13642462,tt0038276,1.0,1.0
3,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt13642594,tt0038276,NaN,NaN
4,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt13918922,tt0038276,NaN,NaN


* Δεν χρειαζόμαστε και το `tconst_ti` και το `parentTconst`, άρα θα κρατήσουμε το ένα από τα δύο.

* Επίσης θα μετονομάσουμε το`tconst_ep` απλώς σε `tconst`.

In [17]:
titles_episodes_df = titles_episodes_df.drop('tconst_ti', axis=1)
titles_episodes_df.rename(columns={'tconst_ep': 'tconst'}, inplace=True)
titles_episodes_df.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tconst,parentTconst,seasonNumber,episodeNumber
0,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt12257020,tt0038276,NaN,NaN
1,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt12283504,tt0038276,NaN,NaN
2,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt13642462,tt0038276,1.0,1.0
3,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt13642594,tt0038276,NaN,NaN
4,tvSeries,You Are an Artist,You Are an Artist,0.0,1946.0,1955.0,15.0,Talk-Show,tt13918922,tt0038276,NaN,NaN


* Στην πραγματικότητα, έχουμε περισσότερα δεδομένα από όσα χρειαζόμαστε.

* Μπορούμε να αφαιρέσουμε κάποιες από τις στήλες που δεν θα χρησιμοποιήσουμε.

In [18]:
titles_episodes_df = titles_episodes_df.drop(['titleType', 
                                              'originalTitle',
                                              'isAdult', 
                                              'runtimeMinutes',
                                              'genres'], axis=1)
titles_episodes_df.head()

,primaryTitle,startYear,endYear,tconst,parentTconst,seasonNumber,episodeNumber
0,You Are an Artist,1946.0,1955.0,tt12257020,tt0038276,NaN,NaN
1,You Are an Artist,1946.0,1955.0,tt12283504,tt0038276,NaN,NaN
2,You Are an Artist,1946.0,1955.0,tt13642462,tt0038276,1.0,1.0
3,You Are an Artist,1946.0,1955.0,tt13642594,tt0038276,NaN,NaN
4,You Are an Artist,1946.0,1955.0,tt13918922,tt0038276,NaN,NaN


* Για όσες σειρές δεν έχουμε `endYear` θα χρησιμοποιήσουμε την τρέχουσα χρονιά.

In [19]:
import datetime
cur_year = int(datetime.datetime.now().year)

titles_episodes_df['endYear'].fillna(cur_year, inplace=True)

* Υπάρχουν περιπτώσεις χωρίς`seasonNumber`, όπου θα το θέσουμε ίσο με 1.

In [20]:
titles_episodes_df['seasonNumber'].fillna(1, inplace=True)

titles_episodes_df.head()

,primaryTitle,startYear,endYear,tconst,parentTconst,seasonNumber,episodeNumber
0,You Are an Artist,1946.0,1955.0,tt12257020,tt0038276,1.0,NaN
1,You Are an Artist,1946.0,1955.0,tt12283504,tt0038276,1.0,NaN
2,You Are an Artist,1946.0,1955.0,tt13642462,tt0038276,1.0,1.0
3,You Are an Artist,1946.0,1955.0,tt13642594,tt0038276,1.0,NaN
4,You Are an Artist,1946.0,1955.0,tt13918922,tt0038276,1.0,NaN


* Στη συνέχεια, διαβάζουμε τις κριτικές.

In [21]:
ratings_df = pd.read_csv("tvseries/title.ratings.tsv.gz", 
                         dtype=column_types,
                         na_values=r'\N',
                         sep="\t")
print(ratings_df.shape)
ratings_df.head()

(1213727, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1858
1,tt0000002,6.0,243
2,tt0000003,6.5,1627
3,tt0000004,6.0,157
4,tt0000005,6.2,2455


* Θα ενώσουμε και αυτό το `DataFrame` με την τρέχουσα κατάσταση.

In [22]:
titles_episodes_rankings_df = pd.merge(titles_episodes_df, 
                                       ratings_df, 
                                       on='tconst')
titles_episodes_rankings_df.head()

,primaryTitle,startYear,endYear,tconst,parentTconst,seasonNumber,episodeNumber,averageRating,numVotes
0,Actor's Studio,1948.0,1950.0,tt0505144,tt0040021,2.0,15.0,8.5,24
1,Actor's Studio,1948.0,1950.0,tt0505147,tt0040021,1.0,16.0,8.5,25
2,Actor's Studio,1948.0,1950.0,tt0505160,tt0040021,2.0,20.0,6.8,8
3,Actor's Studio,1948.0,1950.0,tt0505175,tt0040021,2.0,29.0,6.8,9
4,The Philco Television Playhouse,1948.0,1956.0,tt0245266,tt0040049,5.0,23.0,8.0,307


* Γνωρίζουμε πότε ξεκίνησε και πότε τελείωσε κάθε σειρά.

* Δεν γνωρίζουμε πότε παίχτηκε κάθε κύκλος μιας σειράς.

* Για να το βρούμε αυτό, θα ξεκινήσουμε με τον αριθμό των κύκλων κάθε σειράς.

In [23]:
seasons = titles_episodes_df[['parentTconst', 'seasonNumber']]\
    .groupby(['parentTconst']).max()
seasons.reset_index(inplace=True)
seasons.head()

,parentTconst,seasonNumber
0,tt0038276,1.0
1,tt0039120,1.0
2,tt0039122,1.0
3,tt0039124,1.0
4,tt0039125,1.0


* Θα μετονομάσουμε τη στήλη ώστε να είναι φανερό ότι πρόκειται για τον αριθμό των κύκλων (= ο μέγιστος κύκλος που έχει η σειρά).

In [24]:
seasons.rename(columns={'seasonNumber' : 'numSeasons'}, inplace=True)
seasons.head()

,parentTconst,numSeasons
0,tt0038276,1.0
1,tt0039120,1.0
2,tt0039122,1.0
3,tt0039124,1.0
4,tt0039125,1.0


* Ενώνουμε τα πάντα.

In [25]:
titles_episodes_rankings_df = pd.merge(titles_episodes_rankings_df, seasons, on='parentTconst')
titles_episodes_rankings_df.head()

,primaryTitle,startYear,endYear,tconst,parentTconst,seasonNumber,episodeNumber,averageRating,numVotes,numSeasons
0,Actor's Studio,1948.0,1950.0,tt0505144,tt0040021,2.0,15.0,8.5,24,2.0
1,Actor's Studio,1948.0,1950.0,tt0505147,tt0040021,1.0,16.0,8.5,25,2.0
2,Actor's Studio,1948.0,1950.0,tt0505160,tt0040021,2.0,20.0,6.8,8,2.0
3,Actor's Studio,1948.0,1950.0,tt0505175,tt0040021,2.0,29.0,6.8,9,2.0
4,The Philco Television Playhouse,1948.0,1956.0,tt0245266,tt0040049,5.0,23.0,8.0,307,8.0


* Για να βρούμε τη χρονιά του κάθε κύκλου, θα υποθέσουμε ότι οι κύκλοι έβγαιναν στον αέρα ανά ίσα διαστήματα από την αρχή μέχρι το τέλος μιας σειράς.

In [26]:
titles_episodes_rankings_df['year'] = titles_episodes_rankings_df.apply(
    lambda x: np.linspace(int(x['startYear']), 
                          int(x['endYear']), 
                          int(x['numSeasons']))[int(x['seasonNumber']) - 1],
    axis=1
)

* Να τι παίρνουμε:

In [27]:
titles_episodes_rankings_df.query('primaryTitle == "Game of Thrones"')[::8]

,primaryTitle,startYear,endYear,tconst,parentTconst,seasonNumber,episodeNumber,averageRating,numVotes,numSeasons,year
246302,Game of Thrones,2011.0,2019.0,tt1480055,tt0944947,1.0,1.0,9.1,45634,8.0,2011.000000
246310,Game of Thrones,2011.0,2019.0,tt1851397,tt0944947,1.0,10.0,9.5,37514,8.0,2011.000000
246318,Game of Thrones,2011.0,2019.0,tt2085238,tt0944947,2.0,6.0,9.1,27958,8.0,2012.142857
246326,Game of Thrones,2011.0,2019.0,tt2178796,tt0944947,3.0,10.0,9.2,30348,8.0,2013.285714
246334,Game of Thrones,2011.0,2019.0,tt2972426,tt0944947,4.0,3.0,8.9,28863,8.0,2014.428571
246342,Game of Thrones,2011.0,2019.0,tt3658012,tt0944947,5.0,1.0,8.5,29578,8.0,2015.571429
246350,Game of Thrones,2011.0,2019.0,tt3866846,tt0944947,5.0,7.0,9.0,28844,8.0,2015.571429
246358,Game of Thrones,2011.0,2019.0,tt4283060,tt0944947,6.0,7.0,8.6,32155,8.0,2016.714286
246366,Game of Thrones,2011.0,2019.0,tt5775854,tt0944947,7.0,5.0,8.8,43129,8.0,2017.857143
246374,Game of Thrones,2011.0,2019.0,tt6027920,tt0944947,8.0,6.0,4.0,241077,8.0,2019.000000


* Μπορούμε λοιπόν να ομαδοποιήσουμε ανά σειρά και κύκλο ώστε να βρούμε τη μέση βαθμολογία και τον αριθμό κριτικών.

In [28]:
to_show = titles_episodes_rankings_df[
    ['parentTconst', 
     'primaryTitle', 
     'seasonNumber', 
     'year',
     'startYear',
     'numSeasons',
     'averageRating', 
     'numVotes']]\
    .groupby(['parentTconst', 'primaryTitle', 'seasonNumber', 'year', 'numSeasons', 'startYear'])\
    .agg({'averageRating': 'mean',
          'numVotes': 'sum'}).reset_index()
print(to_show.shape)
to_show.head()

(50079, 8)


,parentTconst,primaryTitle,seasonNumber,year,numSeasons,startYear,averageRating,numVotes
0,tt0040021,Actor's Studio,1.0,1948.000000,2.0,1948.0,8.500000,25
1,tt0040021,Actor's Studio,2.0,1950.000000,2.0,1948.0,7.366667,41
2,tt0040049,The Philco Television Playhouse,1.0,1948.000000,8.0,1948.0,6.700000,9
3,tt0040049,The Philco Television Playhouse,3.0,1950.285714,8.0,1948.0,7.800000,9
4,tt0040049,The Philco Television Playhouse,5.0,1952.571429,8.0,1948.0,7.650000,316


* Επειδή δεν θέλουμε να βλέπουμε σκουπίδια, θα κρατήσουμε τις σειρές με βαθμολογία από 5 και πάνω.

* Και για μεγαλύτερη ευκρίνεια στο διάγραμμα, θα κρατήσουμε τις σειρές από το 1990 μέχρι το 2021.

In [29]:
to_show = to_show.query('(averageRating >= 5) & (startYear >= 1990) & (year <= 2021)')
print(to_show.shape)
to_show.head()

(37322, 8)


,parentTconst,primaryTitle,seasonNumber,year,numSeasons,startYear,averageRating,numVotes
4494,tt0088655,AD Police Files,1.0,1990.000000,1.0,1990.0,6.800000,363
4504,tt0089749,Otaku no video,1.0,1991.000000,1.0,1991.0,7.050000,17
5097,tt0094547,Sidewalks Entertainment,17.0,2008.451613,32.0,1994.0,8.600000,64
5148,tt0095670,La mujer de tu vida,1.0,1990.000000,1.0,1990.0,5.971429,91
5289,tt0096565,The Detectives,1.0,1993.000000,5.0,1993.0,8.100000,187


* Επιπλέον, θα κρατήσουμε μόνο τις σειρές που είχαν κατά μέσο όρο από 1000 κριτικές και πάνω ανά κύκλο.er season.

* Πρώτα εντοπίζουμε τις σειρές που πληρούν το κριτήριο αυτό.

In [30]:
avg_votes_gt_1000 = to_show.groupby('parentTconst').agg({'numVotes': 'mean'}).query('numVotes >= 1000')
print(avg_votes_gt_1000.shape)
avg_votes_gt_1000.head()

(2693, 1)


,numVotes
parentTconst,
tt0096657,23594.000000
tt0098749,3946.200000
tt0098765,2454.666667
tt0098769,3030.000000
tt0098798,4389.000000


* Μετά χρησιμοποιούμε τις σειρές αυτές ως φίλτρο μέσω μιας ένωσης από δεξιά (right join).

In [31]:
to_show = pd.merge(to_show, avg_votes_gt_1000, how='right', left_on='parentTconst', right_index=True)
to_show.rename(columns={'numVotes_x': 'numVotes', 'numVotes_y': 'avgVotes'}, inplace=True)
print(to_show.shape)
to_show.head()

(7106, 9)


,parentTconst,primaryTitle,seasonNumber,year,numSeasons,startYear,averageRating,numVotes,avgVotes
5372,tt0096657,Mr. Bean,1.0,1990.000000,1.0,1990.0,8.680000,23594,23594.0
5504,tt0098749,"Beverly Hills, 90210",1.0,1990.000000,10.0,1990.0,6.786364,6204,3946.2
5505,tt0098749,"Beverly Hills, 90210",2.0,1991.111111,10.0,1990.0,6.692857,5536,3946.2
5506,tt0098749,"Beverly Hills, 90210",3.0,1992.222222,10.0,1990.0,6.733333,4838,3946.2
5507,tt0098749,"Beverly Hills, 90210",4.0,1993.333333,10.0,1990.0,6.609677,4388,3946.2


* Θα πρέπει να βρούμε το ποσοστό των κριτικών από όλες τις κριτικές της χρονιάς που έλαβε κάθε συγκεκριμένη σειρά. 

* Για να το κάνουμε αυτό θα πρέπει να κάνουμε τα έτη ακέραιους αριθμούς.

In [32]:
to_show['intYear'] = to_show['year'].astype(int)
to_show.head()

,parentTconst,primaryTitle,seasonNumber,year,numSeasons,startYear,averageRating,numVotes,avgVotes,intYear
5372,tt0096657,Mr. Bean,1.0,1990.000000,1.0,1990.0,8.680000,23594,23594.0,1990
5504,tt0098749,"Beverly Hills, 90210",1.0,1990.000000,10.0,1990.0,6.786364,6204,3946.2,1990
5505,tt0098749,"Beverly Hills, 90210",2.0,1991.111111,10.0,1990.0,6.692857,5536,3946.2,1991
5506,tt0098749,"Beverly Hills, 90210",3.0,1992.222222,10.0,1990.0,6.733333,4838,3946.2,1992
5507,tt0098749,"Beverly Hills, 90210",4.0,1993.333333,10.0,1990.0,6.609677,4388,3946.2,1993


* Και στη συνέχεια να ομαδοποιήσουμε κατά έτος.

In [33]:
votes_per_year = to_show[['intYear', 'numVotes']].groupby('intYear').sum()
votes_per_year.rename(columns={'numVotes': 'yearVotes'}, inplace=True)
votes_per_year.head()

,yearVotes
intYear,
1990,175285
1991,174019
1992,180251
1993,272075
1994,393316


In [34]:
votes_per_year.tail()

,yearVotes
intYear,
2017,5552744
2018,4837563
2019,6250598
2020,4551131
2021,2708228


* Έτσι, μπορούμε να βρούμε το μερίδιο των κριτικών που έλαβε κάθε σειρά κάθε χρονιά.

In [35]:
to_show = pd.merge(to_show, votes_per_year, left_on='intYear', right_index=True)
to_show['propVotes'] = 100 * to_show['numVotes'] / to_show['yearVotes']
to_show.sample(n=10, random_state=42)

,parentTconst,primaryTitle,seasonNumber,year,numSeasons,startYear,averageRating,numVotes,avgVotes,intYear,yearVotes,propVotes
35012,tt2618986,Wayward Pines,2.0,2016.0,2.0,2015.0,7.390000,8068,13743.500000,2016,5701767,0.141500
18290,tt0758790,The Tudors,2.0,2008.0,4.0,2007.0,8.180000,6102,5828.500000,2008,2317399,0.263312
14229,tt0353049,Chappelle's Show,1.0,2003.0,3.0,2003.0,7.885714,6699,4708.000000,2003,963117,0.695554
26090,tt1358522,White Collar,3.0,2011.0,6.0,2009.0,8.037500,11572,10146.166667,2011,3910908,0.295890
15940,tt0417373,The Venture Bros.,3.0,2008.0,7.0,2003.0,8.492308,2824,2731.285714,2008,2317399,0.121861
31016,tt1839578,Person of Interest,1.0,2011.0,5.0,2011.0,8.804348,78459,68028.400000,2011,3910908,2.006158
34762,tt2520512,Maron,2.0,2014.0,4.0,2013.0,8.138462,1188,1087.250000,2014,4986626,0.023824
7731,tt0115320,The Pretender,1.0,1996.0,4.0,1996.0,8.004545,3138,2632.750000,1996,514665,0.609717
17270,tt0460637,Everybody Hates Chris,1.0,2005.0,4.0,2005.0,7.759091,5006,3978.500000,2005,1902889,0.263074
41690,tt5212822,Imposters,1.0,2017.0,2.0,2017.0,8.200000,3144,2459.000000,2017,5552744,0.056621


* Με όλα αυτά, μπορούμε να φτιάξουμε ένα διάγραμμα.

* Θα έχουμε ένα σημείο για κάθε σειρά.

* Στον οριζόντιο άξονα θα έχουμε τη σειρά και στον κάθετο τη βαθμολογία.

* Το χρώμα κάθε σημείου θα προκύπτει από τον αριθμό ψήφων κάθε σειράς σε κάθε κύκλο.

In [36]:
size = to_show['numVotes'].values
size = np.log(size)

text = (to_show['primaryTitle'].map(str) 
        + " " 
        + to_show['seasonNumber'].astype(int).map(str)
        + " "
        + to_show['propVotes'].map(str))

fig = go.Figure()

trace = go.Scattergl(
    x = to_show['year'],
    y = to_show['averageRating'],
    text=text,
    mode = 'markers',
    hoverinfo="y+text",
    marker=dict(
        color=size,
        colorscale='magma_r',
        size=10,
        sizemode='area',
    ),
    opacity=0.75)

fig.add_trace(trace)
fig.update_layout(template='plotly_white')
plotly.offline.plot(fig, "tvseries.html")
fig.show()

* Εναλλακτικά, μπορούμε να φτιάξουμε το διάγραμμα με το Bokeh.

* Εδώ το μέγεθος κάθε σειράς θα είναι ανάλογο με το ποσοστό κριτικών που έχει λάβει κάθε κύκλος της σειράς.

In [37]:
import bokeh.plotting as bk
import matplotlib as mpl
import matplotlib.pyplot as plt

from bokeh.plotting import figure
from bokeh.layouts import layout, column
from bokeh.transform import log_cmap
from bokeh.models import ColumnDataSource, CustomJS, HoverTool, TapTool
from bokeh.layouts import column
from bokeh.models.glyphs import Line
from bokeh.models.glyphs import Circle

* Επειδή το Bokeh χρησιμοποιεί ως μέθεγος την ακτίνα κάθε κύκλου, θα χρησιμοποιήσουμε την τετραγωνική ρίζα του `propVotes` ως βάση για το μέγεθος.

In [38]:
to_show['visPropVotes'] = 10 * np.sqrt(to_show['propVotes']) 

In [39]:
TOOLTIPS= [
    ("Title", "@primaryTitle"),
    ("Season", "@seasonNumber{0}"),
    ("Year", "@year{0}"),
    ("Ratings", "@numVotes"),
    ("Average Rating", "@averageRating"),
    #("", "<style>.bk-tooltip>div:not(:first-child) {display:none;}</style>")
]

p = figure(plot_height=800, 
           plot_width=1400, 
           title="TV Shows", 
           toolbar_location=None, 
           sizing_mode="scale_both")

source = ColumnDataSource(to_show)
line_source = ColumnDataSource(data=dict(x=[], y=[]))
selected_source = ColumnDataSource(data=dict(x=[], y=[], size=[]))

hover_callback_code = """
var data = source.data;
var line_data = [];
var selected_data = [];
var tconsts = data['parentTconst'];
var tconst = "";
var indices = [];
var hover_indices = cb_data.index['1d'].indices;
var keep_looking = true;
var i = 0;
var indx;

line_data['x'] = [];
line_data['y'] = [];
selected_data['x'] = [];
selected_data['y'] = [];
selected_data['size'] = [];

if (data['numSeasons'] <= 1) {
  line_source.data = line_data;
  selected_source.data = selected_data;
  line_source.change.emit();
  selected_source.change.emit();
  return;
} 

indx = hover_indices[0];
indices.push(indx);
tconst = tconsts[indx];
i = indx;

while (indx != null && keep_looking) {
  i--;
  if (i == 0) {
    keep_looking = false;
  }
  if (tconsts[i] === tconst) {
    indices.push(i);
  }
}

keep_looking = true;
i = indx;
while (indx != null && keep_looking) {
  i++;
  if (i == tconsts.length) {
    keep_looking = false;
  }
  if (tconsts[i] === tconst) {
    indices.push(i);
  }
}

if (indx && indices.length) {

  // Sort the data corresponding to the collected indices
  // chronologically.
  var list = [];
  for (var i = 0; i < indices.length; i++) {
    list.push({
      'year': data['year'][indices[i]], 
      'averageRating': data['averageRating'][indices[i]],
      'visPropVotes': data['visPropVotes'][indices[i]]
    });
  }
  list.sort((a, b) => a.year - b.year);
  for (var i = 0; i < list.length; i++) {
    line_data['x'].push(list[i].year);
    line_data['y'].push(list[i].averageRating);
    selected_data['x'].push(list[i].year);
    selected_data['y'].push(list[i].averageRating);
    selected_data['size'].push(list[i].visPropVotes);
  }
}
line_source.data = line_data;
selected_source.data = selected_data;
selected_source.change.emit();
line_source.change.emit();
""" 

sc = p.scatter(x='year', 
               y='averageRating',
               name='scatter',
               line_color='black',
               color='lightsteelblue',
               hover_color="steelblue",
               size='visPropVotes',
               line_alpha=0.7,
               fill_alpha=0.7,
               source=source)

line_renderer = p.line(x="x", y="y", line_color="steelblue", line_width=3, line_alpha=1, source=line_source)

circles_renderer = p.circle(x="x", y="y", 
                            line_color="steelblue", 
                            fill_color="steelblue",
                            line_width=1, 
                            size="size",
                            source=selected_source)

hover_callback = CustomJS(args=dict(source=sc.data_source,
                                    line_source=line_renderer.data_source,
                                    selected_source=circles_renderer.data_source),
                          code=hover_callback_code)

hover = HoverTool(names=["scatter"], 
                  tooltips=TOOLTIPS, 
                  callback=hover_callback)

p.add_tools(hover)

bk.output_notebook()
bk.output_file('tv_series_bokeh.html')
bk.show(p)

Loading BokehJS ...

* Τέλος, μπορούμε να φτιάξουμε ένα πιο εξελιγμένο διάγραμμα με το [d3js](https://d3js.org/).

* Αυτό θα πρέπει να γίνει εκτός του παρόντος notebook.

* Θα αποθηκεύσουμε το `DataFrame` `to_show` για να το χρησιμοποιήσουμε με το d3js.

In [40]:
to_show.to_csv('to_show.csv', index=False)

* Θα χρειαστούμε και ένα επιπλέον αρχείο το οποίο θα περιέχει τις χρονιές και τις μέσες κριτικές.

* Το αρχείο αυτό θα το χρησιμοποιήσει το d3js για να συνδέσει οπτικά τους κύκλους κάθε σειράς.

In [41]:
lines = to_show[
    [
        'parentTconst', 'seasonNumber', 'year', 'averageRating'
    ]].groupby(['parentTconst'])\
    .agg({'year' : list, 'averageRating': list })


def sort_years_ratings(row):
    arr_years = np.array(row['year'])
    sorted_indices = np.argsort(arr_years)
    arr_years = arr_years[sorted_indices]
    arr_ratings = np.array(row['averageRating'])
    arr_ratings = arr_ratings[sorted_indices]
    row['year'] = arr_years
    row['averageRating'] = arr_ratings
    return row

lines.apply(sort_years_ratings, axis=1)
lines.to_json('lines.json', orient='index')

In [42]:
lines

,year,averageRating
parentTconst,,
tt0096657,[1990.0],[8.68]
tt0098749,"[1990.0, 1991.111111111111, 1992.2222222222222...","[6.786363636363637, 6.692857142857143, 6.73333..."
tt0098765,"[1991.0, 1992.0, 1993.0]","[7.816666666666666, 7.2, 6.6]"
tt0098769,[1990.0],[8.61111111111111]
tt0098798,[1990.0],[7.672727272727273]
...,...,...
tt9860664,[2019.0],[8.45]
tt9861884,[2019.0],[8.872727272727273]
tt9879074,[2019.0],[7.0125]
